In [3]:
#need this to open an excel
pip install openpyxl

     -------------------------------------- 250.0/250.0 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

file_path = r"C:\Users\12039\Documents\DATA 6520\Customer-Segmentation\Online Retail.xlsx"
onlineretail = pd.read_excel(file_path)
onlineretail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Clean Data

#### Handle Missing Data:
- Check for missing values in crucial columns like customer IDs, transaction dates, and purchase amounts.
- Decide on a strategy for dealing with them, such as filling in missing values or removing rows/columns with too many missing values.

#### Remove Duplicates:
- Look for and eliminate duplicate records to ensure the accuracy of your analysis.

#### Filter Out Irrelevant Data:
- Depending on your analysis, you might need to exclude certain types of transactions (like returns or free samples).

#### Data Type Conversions:
- Ensure that each column is of the appropriate data type. For instance, dates should be in a datetime format, customer IDs might be strings, and monetary values should be numeric.

#### Data Range Consistency:
- Check for any anomalies or inconsistencies in data ranges, such as dates in the future or negative monetary values.

#### Creating RFM Metrics:
- Recency: Calculate the number of days since the last purchase for each customer.
- Frequency: Count the number of purchases for each customer.
- Monetary: Calculate the total amount of money each customer has spent.

#### Handling Outliers:
- Look for outliers in your RFM metrics which can skew your analysis. Depending on your dataset and goals, you might want to cap them, remove them, or treat them separately.

#### Normalization or Standardization (If Required):
- Depending on your segmentation method (like K-means clustering), you might need to normalize or standardize the RFM metrics so they're on the same scale.

In [6]:
onlineretail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [14]:
# remove missing CustomerID rows since RFM is all about the customer.
onlineretail_cleaned = onlineretail.dropna(subset=['CustomerID']).copy()
onlineretail_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [10]:
onlinceretail_cleaned.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [15]:
# Change CustomerID to integer instead of float.  
# This will help with memory but mostly avoid ambiguity and data integrity later
onlineretail_cleaned['CustomerID'] = onlineretail_cleaned['CustomerID'].astype(int)

In [17]:
onlineretail_cleaned.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
#Filter our negative or zero values
#There is at least 1 negative quantity